# DOGS VS CATS REDUX: KERNEL EDITION

My model for submission into the the Dogs vs Cats Redux Kaggle Competition:
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

The final model is a finetuned version of a pre-trained VGG16 model which made it into the top 40% of the public leaderboard.

## Imports, helper functions, and data directory definition

In [413]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop

import numpy as np
from sklearn.metrics import log_loss
from matplotlib import pyplot as plt
from shutil import copyfile
%matplotlib inline

In [2]:
def plot_img(img):
    if K.image_dim_ordering() == 'th':
        img = np.rollaxis(img,0,3).astype(np.uint8)
    else:
        img = np.rollaxis(img,0,1).astype(np.uint8)
    plt.imshow(img)

In [211]:
def create_gen(directory, batch_size=4, shuffle=True, 
               gen=ImageDataGenerator(), target_size=(224,224), 
               class_mode = None):
    return gen.flow_from_directory(directory,
                                   batch_size = batch_size,
                                   shuffle = shuffle,
                                   target_size = target_size, 
                                   class_mode = class_mode)

In [412]:
import os,sys
%cd ~/nbs
current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_DIR = HOME_DIR+'/data/dogscats/'

test_dir = DATA_DIR+'test/'
saved_model_dir = DATA_DIR+'saved_models/'

# sample of training data
sample_train_dir = DATA_DIR+'sample/train/'
sample_valid_dir = DATA_DIR+'sample/valid/'

# full training data
train_dir = DATA_DIR+'train/'
valid_dir = DATA_DIR+'valid/'

img_shape = (224,224,3)

/home/ubuntu/nbs


## SPLIT DATA

Get data from here:
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data

I start by splitting the data into testing, training, and validation. I also copy a small random sample from the training set for initial experimentation. These examples are placed in the sample folder.

In [469]:
# Create separate folders for train, valid, and sample/train, sample/valid
%mkdir -p $DATA_DIR/test/test
%mkdir -p $DATA_DIR/train/dogs
%mkdir -p $DATA_DIR/train/cats
%mkdir -p $DATA_DIR/valid/dogs
%mkdir -p $DATA_DIR/valid/cats
%mkdir -p $DATA_DIR/sample/train/dogs
%mkdir -p $DATA_DIR/sample/train/cats
%mkdir -p $DATA_DIR/sample/valid/dogs
%mkdir -p $DATA_DIR/sample/valid/cats
%mkdir -p $saved_model_dir

In [470]:
!tree -d $DATA_DIR

/home/ubuntu/nbs/data/dogscats/
├── sample
│   ├── train
│   │   ├── cats
│   │   └── dogs
│   └── valid
│       ├── cats
│       └── dogs
├── saved_models
├── test
│   └── test
├── train
│   ├── cats
│   └── dogs
└── valid
    ├── cats
    └── dogs

16 directories


In [21]:
# Move training examples to cats/dogs subdirectories
%cd $train_dir
from glob import glob
c = glob('cat.*')
for fname in c: os.rename(fname, DATA_DIR+'/train/cats/'+fname)
d = glob('dog.*')
for fname in d: os.rename(fname, DATA_DIR+'/train/dogs/'+fname)

In [26]:
# Copy some cat and dog examples into the sample folder
%cd $train_dir/cats
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(80): copyfile(shuf[i], DATA_DIR+'/sample/train/cats/'+shuf[i])
for i in range(80,100): copyfile(shuf[i], DATA_DIR+'/sample/valid/cats/'+shuf[i])

In [28]:
%cd $DATA_DIR/train/dogs
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(80): copyfile(shuf[i], DATA_DIR+'/sample/train/dogs/'+shuf[i])
for i in range(80,100): copyfile(shuf[i], DATA_DIR+'/sample/valid/dogs/'+shuf[i])

In [30]:
# Move random subset of training examples to valid directory
%cd $train_dir/cats
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2500): os.rename(shuf[i],DATA_DIR+'/valid/cats/'+shuf[i])

/home/ubuntu/nbs/data/dogscats/train/cats


In [31]:
%cd $train_dir/dogs
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2500): os.rename(shuf[i],DATA_DIR+'/valid/dogs/'+shuf[i])

/home/ubuntu/nbs/data/dogscats/train/dogs


In [427]:
# Create data genererators
batch_size = 64
sample_train_gen = create_gen(sample_train_dir, batch_size=batch_size, shuffle=True, class_mode='binary')
sample_valid_gen = create_gen(sample_valid_dir, batch_size=batch_size, shuffle=True, class_mode='binary')
train_gen = create_gen(train_dir, batch_size=batch_size, shuffle=True, class_mode='binary')
valid_gen = create_gen(valid_dir, batch_size=batch_size, shuffle=True, class_mode='binary')
test_gen = create_gen(test_dir, batch_size=batch_size, shuffle=True, class_mode='binary')

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


## BASELINE MODELS

The first models tried here are meant to serve as benchmarks to compare later models to. The models I try here are Linear, Simple Neural Network, and a Simple CNN.

### Linear Model

This one starts to fit to the data but it doesn't generalizing very well. We can see this from the quick increase in the training accuracy while the validation accuracy doesn't look like it's going anywhere.

In [431]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=img_shape))
model.add(Flatten())
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [439]:
model.fit_generator(sample_train_gen, epochs=4, validation_data=sample_valid_gen, verbose=2)

Epoch 1/4
 - 1s - loss: 0.4671 - acc: 0.7711 - val_loss: 0.6582 - val_acc: 0.5750
Epoch 2/4
 - 1s - loss: 0.4156 - acc: 0.8602 - val_loss: 0.6431 - val_acc: 0.5500
Epoch 3/4
 - 1s - loss: 0.3980 - acc: 0.8734 - val_loss: 0.6370 - val_acc: 0.5500
Epoch 4/4
 - 1s - loss: 0.3557 - acc: 0.9062 - val_loss: 0.6436 - val_acc: 0.6000


### Simple neural network

Same issue as the model above. We can start to overfit it to the data, but it's not generalizing.

In [436]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(224,224,3)))
model.add(Flatten())
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [438]:
model.fit_generator(sample_train_gen, epochs=4, validation_data=sample_valid_gen)

Epoch 1/4
3/3 [==============================] - 1s 376ms/step - loss: 0.5934 - acc: 0.6930 - val_loss: 0.6007 - val_acc: 0.6000
Epoch 2/4
3/3 [==============================] - 0s 166ms/step - loss: 0.5417 - acc: 0.7742 - val_loss: 0.6162 - val_acc: 0.6250
Epoch 3/4
3/3 [==============================] - 1s 205ms/step - loss: 0.5145 - acc: 0.7680 - val_loss: 0.6147 - val_acc: 0.6250
Epoch 4/4
3/3 [==============================] - 1s 201ms/step - loss: 0.4805 - acc: 0.8008 - val_loss: 0.6163 - val_acc: 0.6500


### Simple CNN

The shallow CNN has the same issue as the last two. I try making it deeper, but it doesn't look like it's enough for it to learn anything useful. <br>Next I'll try a pre-trained model.

In [449]:
cnn1 = Sequential()
cnn1.add(BatchNormalization(axis=1, input_shape = img_shape))
cnn1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn1.add(MaxPooling2D((2, 2), strides=(2, 2)))
cnn1.add(Flatten())
cnn1.add(Dense(100))
cnn1.add(BatchNormalization())
cnn1.add(Dense(1,activation = 'sigmoid'))
cnn1.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [450]:
cnn1.fit_generator(sample_train_gen, epochs=4, validation_data=sample_valid_gen)

Epoch 1/4
3/3 [==============================] - 9s 3s/step - loss: 2.4746 - acc: 0.6070 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/4
3/3 [==============================] - 9s 3s/step - loss: 1.8874 - acc: 0.7078 - val_loss: 6.2612 - val_acc: 0.6000
Epoch 3/4
3/3 [==============================] - 9s 3s/step - loss: 1.3439 - acc: 0.7703 - val_loss: 8.6762 - val_acc: 0.4500
Epoch 4/4
3/3 [==============================] - 9s 3s/step - loss: 0.8355 - acc: 0.7555 - val_loss: 6.3318 - val_acc: 0.5250


In [451]:
cnn1 = Sequential()
cnn1.add(BatchNormalization(axis=1, input_shape = img_shape))
# Block 1
cnn1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn1.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn1.add(MaxPooling2D((2, 2), strides=(2, 2)))
# Block 2
cnn1.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn1.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn1.add(MaxPooling2D((2, 2), strides=(2, 2)))
# Block 4
cnn1.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
cnn1.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
cnn1.add(MaxPooling2D((2, 2), strides=(2, 2)))
# Top
cnn1.add(Flatten())
cnn1.add(Dense(256))
cnn1.add(BatchNormalization())
cnn1.add(Dense(1,activation = 'sigmoid'))
cnn1.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [452]:
cnn1.fit_generator(sample_train_gen, epochs=4, validation_data=sample_valid_gen)

Epoch 1/4
3/3 [==============================] - 15s 5s/step - loss: 2.9895 - acc: 0.6008 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/4
3/3 [==============================] - 14s 5s/step - loss: 3.4410 - acc: 0.6781 - val_loss: 7.5753 - val_acc: 0.5000
Epoch 3/4
3/3 [==============================] - 14s 5s/step - loss: 2.9601 - acc: 0.6781 - val_loss: 1.3932 - val_acc: 0.4500
Epoch 4/4
3/3 [==============================] - 14s 5s/step - loss: 2.1013 - acc: 0.6969 - val_loss: 2.8407 - val_acc: 0.5500


## FINETUNED VGG16 MODELS

### Create VGG16 pretrained model

Create VGG16 model and load it with weights trained from imagenet.

In [455]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
model = VGG16(weights='imagenet', include_top=False)

In [456]:
batch_size=64
nb_epochs=3
nb_sample_train = 160
nb_sample_valid = 40
nb_train = 20000
nb_valid = 5000

bn_feat_sample_train = 'bn_feat_sample_train.npy'
bn_feat_sample_valid = 'bn_feat_sample_valid.npy'
bn_feat_train = 'bn_feat_train.npy'
bn_feat_valid = 'bn_feat_valid.npy'
bn_feat_test = 'bn_feat_test.npy'

### Compute VGG16 bottleneck features

Here's where I compute the bottleneck features. These are the outputs from the last convolutional block of VGG16 which would feed into the dense layers on top (which we didn't include). The VGG16 convolutional layers are very good at pulling out useful features from images. With the top included, VGG16 would then take these features, and output an imagenet class. So the convolutional layers pull out useful features from images, and the dense layers use these features to determine what the image is. We're going to keep these features that the VGG16 pulls out of our dataset, and then use those to train a new network to classify them as either cats or dogs.

In [ ]:
# Create generators

datagen = ImageDataGenerator(rescale=1., featurewise_center=True) #(rescale=1./255)
datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3) #3,1,1

sample_train_gen = create_gen(sample_train_dir, batch_size=batch_size, shuffle=False, gen=datagen)
sample_valid_gen = create_gen(sample_valid_dir, batch_size=batch_size, shuffle=False, gen=datagen)
train_gen = create_gen(train_dir, batch_size=batch_size, shuffle=False, gen=datagen)
valid_gen = create_gen(valid_dir, batch_size=batch_size, shuffle=False, gen=datagen)
test_gen = create_gen(test_dir, batch_size=batch_size, shuffle=False, gen=datagen)

In [75]:
# create sample train data bottleneck features
bottleneck_features_train = model.predict_generator(sample_train_gen)
np.save(open(saved_model_dir + bn_feat_sample_train, 'w'),
        bottleneck_features_train)

In [77]:
# create sample validation data bottleneck features
bottleneck_features_valid = model.predict_generator(sample_valid_gen)
np.save(open(saved_model_dir + bn_feat_sample_valid, 'w'),bottleneck_features_valid)

In [ ]:
# create train data bottleneck features
bottleneck_features_train = model.predict_generator(test_gen)
np.save(open(saved_model_dir + bn_feat_train, 'w'),bottleneck_features_train)

In [232]:
# create valid data bottleneck features
bottleneck_features_valid = model.predict_generator(valid_gen)
np.save(open(saved_model_dir + bn_feat_valid, 'w'),bottleneck_features_valid)

Found 5000 images belonging to 2 classes.


In [233]:
# create test data bottleneck features
bottleneck_features_test = model.predict_generator(test_gen)
np.save(open(saved_model_dir + bn_feat_test, 'w'), bottleneck_features_test)

Found 12500 images belonging to 1 classes.


### Load VGG16 features

Load the saved VGG features. There are two cells here, one for training on the full training set, and one for training only on the small sample data set.

In [234]:
# Full training set of features and labels
test_features = np.load(open(saved_model_dir + bn_feat_test))
train_features = np.load(open(saved_model_dir + bn_feat_train))
valid_features = np.load(open(saved_model_dir + bn_feat_valid))
train_labels = np.array([1] * int(nb_train / 2) + [0] * int(nb_train / 2))
valid_labels = np.array([1] * int(nb_valid / 2) + [0] * int(nb_valid / 2))

In [22]:
# # Small sample set of features and labels
# train_features = np.load(open(saved_model_dir + bn_feat_sample_train))
# valid_features = np.load(open(saved_model_dir + bn_feat_sample_valid))
# train_labels = np.array([1] * int(nb_sample_train / 2) + [0] * int(nb_sample_train / 2))
# valid_labels = np.array([1] * int(nb_sample_valid / 2) + [0] * int(nb_sample_valid / 2))

### Linear model on VGG16

This model is just a single linear layer. 

In [457]:
model_fc1 = Sequential()
model_fc1.add(Flatten(input_shape=train_features.shape[1:]))
model_fc1.add(Dense(1, activation='sigmoid'))

model_fc1.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

In [458]:
fc1_weights = 'best_weights_fc1.h5'
checkpointer = ModelCheckpoint(filepath= saved_model_dir + fc1_weights, 
                               save_best_only=True)

model_fc1.fit(train_features,train_labels,
              epochs=5,#nb_epochs,
              batch_size=batch_size,
              validation_data=(valid_features,valid_labels),
              callbacks=[checkpointer],
              verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 2s 123us/step - loss: 0.6293 - acc: 0.9494 - val_loss: 0.6063 - val_acc: 0.9528
Epoch 2/5
20000/20000 [==============================] - 2s 111us/step - loss: 0.3472 - acc: 0.9748 - val_loss: 0.3820 - val_acc: 0.9720
Epoch 3/5
20000/20000 [==============================] - 2s 110us/step - loss: 0.3231 - acc: 0.9762 - val_loss: 0.4282 - val_acc: 0.9690
Epoch 4/5
20000/20000 [==============================] - 2s 111us/step - loss: 0.2989 - acc: 0.9786 - val_loss: 0.3564 - val_acc: 0.9744
Epoch 5/5
20000/20000 [==============================] - 2s 108us/step - loss: 0.2677 - acc: 0.9811 - val_loss: 0.3521 - val_acc: 0.9742


In [465]:
# Log loss on validation set
model_fc1.load_weights(saved_model_dir + fc1_weights)
pred = model_fc1.predict(valid_features)
pred = pred.reshape(pred.shape[0])
predictions = pred.clip(min=0.05, max=0.95)
log_loss(valid_labels,predictions)

0.12561574509888887

### Final model on VGG16

In [460]:
model_fc3 = Sequential()
model_fc3.add(MaxPooling2D((2, 2), strides=(2, 2), input_shape=train_features.shape[1:]))
model_fc3.add(Flatten())
model_fc3.add(Dense(4096, activation='relu'))
model_fc3.add(BatchNormalization())
model_fc3.add(Dropout(0.5))
model_fc3.add(Dense(4096, activation='relu'))
model_fc3.add(BatchNormalization())
model_fc3.add(Dropout(0.5))
model_fc3.add(Dense(1, activation='sigmoid'))

model_fc3.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

In [461]:
fc3_weights = 'best_weights_fc3.h5'
checkpointer = ModelCheckpoint(filepath= saved_model_dir + fc3_weights, 
                               save_best_only=True)

model_fc3.fit(train_features,train_labels,
              epochs=5,#nb_epochs,
              batch_size=batch_size,
              validation_data=(valid_features,valid_labels),
              callbacks=[checkpointer],
              verbose=1)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 14s 693us/step - loss: 0.2223 - acc: 0.9534 - val_loss: 0.1175 - val_acc: 0.9702
Epoch 2/5
20000/20000 [==============================] - 14s 697us/step - loss: 0.0821 - acc: 0.9733 - val_loss: 0.0818 - val_acc: 0.9752
Epoch 3/5
20000/20000 [==============================] - 12s 587us/step - loss: 0.0543 - acc: 0.9829 - val_loss: 0.0824 - val_acc: 0.9778
Epoch 4/5
20000/20000 [==============================] - 12s 586us/step - loss: 0.0373 - acc: 0.9887 - val_loss: 0.1124 - val_acc: 0.9766
Epoch 5/5
20000/20000 [==============================] - 12s 585us/step - loss: 0.0350 - acc: 0.9904 - val_loss: 0.0892 - val_acc: 0.9782


Doesn't look like we're seeing much improvement on the loss function after a few epochs. We'd probably need to tweak some hyperparameters or the architecture to improve much from here.

In [464]:
# Log loss on validation set
model_fc3.load_weights(saved_model_dir + fc3_weights)
pred = model_fc3.predict(valid_features)
pred = pred.reshape(pred.shape[0])
predictions = pred.clip(min=0.05, max=0.95)
log_loss(valid_labels,predictions)

0.10920777253806591

Looks like it's an improvement on the linear model above. It's surprising how well the linear model did though. Let's try an ensemble of this model next.

## ENSEMBLE

In [466]:
class Ensemble():
    
    def __init__(self, nb_models = 3):
        self.nb_models = nb_models
    
    def create_model(self):
        model = Sequential()
        model.add(MaxPooling2D((2, 2), strides=(2, 2), input_shape=train_features.shape[1:]))
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def train(self,epochs=3):
        for i in range(self.nb_models):
            
            weights = 'best_weights_fc3_'+str(i)+'.h5'
            checkpointer = ModelCheckpoint(filepath= saved_model_dir + weights, 
                                           save_best_only=True)
            model = self.create_model()
            model.fit(train_features,train_labels,
                          epochs=epochs,
                          batch_size=batch_size,
                          validation_data=(valid_features,valid_labels),
                          callbacks=[checkpointer],
                          verbose=1)
            
    def predict(self, x):
        preds = []
        model = self.create_model()
        for i in range(self.nb_models):
            model.load_weights(saved_model_dir + 'best_weights_fc3_'+str(i)+'.h5')
            preds.append(model.predict(x))
        ens_pred = np.stack(preds).mean(axis=0)
        return ens_pred

In [390]:
model_ens = Ensemble(3)
model_ens.train(3)

In [396]:
# Log log on validation set
ens_pred = model_ens.predict(valid_features)
ens_pred = ens_pred.reshape(ens_pred.shape[0])
predictions = ens_pred.clip(min=0.05, max=0.95)
log_loss(valid_labels,predictions)

0.097072397928684953

In [467]:
model_ens = Ensemble(5)
model_ens.train(3)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 14s 687us/step - loss: 0.2133 - acc: 0.9523 - val_loss: 0.0934 - val_acc: 0.9748
Epoch 2/3
20000/20000 [==============================] - 12s 575us/step - loss: 0.0794 - acc: 0.9747 - val_loss: 0.1409 - val_acc: 0.9692
Epoch 3/3
20000/20000 [==============================] - 14s 700us/step - loss: 0.0523 - acc: 0.9847 - val_loss: 0.0838 - val_acc: 0.9776
Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 14s 680us/step - loss: 0.2159 - acc: 0.9535 - val_loss: 0.1117 - val_acc: 0.9740
Epoch 2/3
20000/20000 [==============================] - 14s 691us/step - loss: 0.0863 - acc: 0.9727 - val_loss: 0.0786 - val_acc: 0.9756
Epoch 3/3
20000/20000 [==============================] - 12s 581us/step - loss: 0.0552 - acc: 0.9824 - val_loss: 0.0822 - val_acc: 0.9784
Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==

In [468]:
# Log log on validation set
ens_pred = model_ens.predict(valid_features)
ens_pred = ens_pred.reshape(ens_pred.shape[0])
predictions = ens_pred.clip(min=0.05, max=0.95)
log_loss(valid_labels,predictions)

0.0926154362514615

As usual, an ensemble tends to do better. Adding more models seems to continue the trend. We could also try an ensemble of different types of models, perhaps using different pre-trained model architectures. I think this should be enough to get into the top 40% of the kaggle competition though.

## PREPARE KAGGLE SUBMISSION

In [407]:
# Make predictions on test set
iscat_pred = model_ens.predict(test_features)
predictions = (1 - iscat_pred).reshape(iscat_pred.shape[0])
predictions = predictions.clip(min=0.05, max=0.95)

In [404]:
# Get file indices
generator = create_gen(test_dir, batch_size=batch_size, shuffle=False, gen=datagen)
filenames = generator.filenames
idx = np.array([int(f[5:f.rfind('.')]) for f in filenames])

Found 12500 images belonging to 1 classes.


In [408]:
# Save predictions
subm = np.stack([idx,predictions],axis=1)
submission_file_name = 'submission3.csv'
%cd ~/git/dogs-v-cats-redux
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

/home/ubuntu/git/dogs-v-cats-redux
